In [4]:
import pandas as pd
import numpy as np
from itertools import combinations 
import networkx as nx
import time

In [5]:
title_akas = pd.read_csv('title.akas.tsv', sep='\t', index_col = 'titleId', usecols=['titleId', 'language','title', 'isOriginalTitle'], low_memory=False)
req = (set(title_akas[(title_akas['language']=='en')].index))
title_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False, 
                           usecols=['tconst','titleType', 'primaryTitle', 'genres'], index_col= 'tconst')



In [6]:
#Featurize genres
node_data = title_basics.reindex(req)
node_data['genres'] = node_data['genres'].str.split(',')   #Convert genres to lists
node_data.dropna(inplace=True)   #Remove the one row with no data
genres_edge = pd.Series(node_data['genres']).apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*genres_edge.genre):
    genres_edge[genre] = genres_edge.apply(lambda _: int(genre in _.genre), axis=1)


In [7]:
#Create Graph and add nodes
G = nx.Graph()
for index,row in node_data.iterrows():
    G.add_node(index, data=row['primaryTitle'])

In [8]:
genres_for_edges = [ 'Game-Show',
 'Talk-Show',
 'Reality-TV',
 'Film-Noir',
 'News',
 'Western',
 'Sport',
 'Musical',
 'Music',
 'Adult',
 'Biography',
 'Sci-Fi',
 'War',
 'History',
'Mystery',
 'Fantasy',
 'Family',
 'Horror',
 'Thriller',
 'Animation',
 'Adventure',
 'Crime',
 '\\N',
 'Romance',
 'Action',
 'Documentary',
 'Short',
 'Comedy',
 'Drama']
num=0
edges = set()
for a in genres_for_edges:
    t0 = time.time()
    for x,y in combinations(genres_edge[genres_edge[a]==1].index, 2):
        num += 1
        edges.add((x,y))
    t1 = time.time()
    print (a, ":" ,sum(genres_edge[a]), ":", t1-t0, ':', t1)
    
print (len(G.edges))
# num_unique_edges = len(edges) #Number of unique edges
# print (num_combinations - num_unique_edges)
# print (num_unique_edges)
a = len(node_data.index)
print (a * (a - 1)) #Max number of edges possible in the graph

Game-Show : 130 : 0.03874778747558594 : 1548989837.7415638
Talk-Show : 247 : 0.01789093017578125 : 1548989837.762362
Reality-TV : 282 : 0.021636247634887695 : 1548989837.786765
Film-Noir : 330 : 0.03308606147766113 : 1548989837.82287
News : 382 : 0.0500490665435791 : 1548989837.875744
Western : 1036 : 0.3408470153808594 : 1548989838.219768
Sport : 1087 : 0.33145594596862793 : 1548989838.553934
Musical : 1539 : 0.7358970642089844 : 1548989839.292408
Music : 1764 : 1.0137391090393066 : 1548989840.309074
Adult : 1768 : 1.1691741943359375 : 1548989841.48095
Biography : 2392 : 1.7218470573425293 : 1548989843.2053492
Sci-Fi : 2401 : 2.5328261852264404 : 1548989845.741079
War : 2667 : 2.1435959339141846 : 1548989847.887209
History : 3032 : 3.091870069503784 : 1548989850.981645
Mystery : 3424 : 6.921218156814575 : 1548989857.9054852
Fantasy : 4117 : 5.544727087020874 : 1548989863.4568372
Family : 4258 : 6.326105117797852 : 1548989869.785444
Horror : 4533 : 12.289029836654663 : 1548989882.07741

KeyboardInterrupt: 

In [1]:
genres_edge.head()  #Node data

# for index, row in node_data.iterrows():
    

NameError: name 'genres_edge' is not defined

In [13]:
list(edges)[:5] #Edge data

[]

In [7]:
#Begin Graph

for x in list(edges):
    G.add_edge (x[0],x[1])

In [10]:
T = nx.Graph()
T.clear()
T.add_node(1)
T.add_node(2)
T.add_node(3)
T.add_edge(1,2)
T.add_edge(1,2)
T.add_edge(1,3)
len(T.edges)

2